<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>IA Generativa e LLMs Para Processamento de Linguagem Natural</font>
## <font color='blue'>Projeto 1</font>
## <font color='blue'>Converse com Seus PDFs - Criando Assistente Pessoal de IA com LLM e VectorDB</font>

## Instalando e Carregando Pacotes

Para instalar todos os pacotes de uma vez, execute no terminal ou prompt de comando:

pip install -r requirements.txt

In [1]:
!pip install -q -r requirements.txt

In [2]:
!pip install -q -U watermark

In [3]:
# Imports
import os
import openai
import langchain
import chromadb
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
import warnings
warnings.filterwarnings('ignore')

In [4]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



## Extraindo Texto de Arquivos PDF

In [5]:
# Função para ler os arquivos em pdf
def dsa_read_pdf(directory_path):
    
    # Acessa a pasta com o arquivo pdf
    file_loader = PyPDFDirectoryLoader(directory_path)
    
    # Lê o documento da pasta
    documents = file_loader.load()
    
    return documents

In [6]:
# Executa a função
dsa_doc = dsa_read_pdf('arquivos/')

In [7]:
dsa_doc

[Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2023-12-18T06:02:27+00:00', 'author': 'DM PM', 'moddate': '2023-12-18T06:02:27+00:00', 'source': 'arquivos/ArtigoDSA1.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='A Habilidade Mais Importante na Era da Inteligência Artificial \n \nA pandemia do COVID-19 acelerou o ritmo do desenvolvimento digital em todo o mundo, já que \ntudo, desde reuniões até consultas médicas, ficou online. Isso pode soar como algo super \npositivo.  \nPara dezenas de milhões de trabalhadores, não. \nEles talvez não tenham as habilidades necessárias para competir nesse novo mundo. Eles são os \ncontadores, os digitadores, os secretários executivos, procurando trabalho em uma nova \neconomia na qual as pessoas contratadas têm títulos como “Engenheiro de Nuv em” ou “Hacker \nde Crescimento” em seus currículos. Sem um esforço concentrado para retreiná-los, descobriram \nos pesquisadores da RAND Europe, eles 

In [8]:
len(dsa_doc)

3

## Definindo a API do LLM

Crie sua API na OpenAI e coloque no arquivo .env na mesma pasta onde está este Jupyter Notebook.

https://platform.openai.com/

https://platform.openai.com/api-keys

https://platform.openai.com/docs/quickstart?context=python
<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
Veja mais detalhes no videobook do Capítulo 5 do curso.

In [9]:
# Carregando as variáveis de ambiente definidas no arquivo .env
from dotenv import load_dotenv
load_dotenv()

True

## Criando o Gerador de Embeddings

In [10]:
# Criando o gerador de embeddings
gerador_embeddings = OpenAIEmbeddings(api_key = os.environ['OPENAI_API_KEY'])

O OpenAIEmbeddings é uma funcionalidade que permite obter representações vetoriais (embeddings) de texto, que são úteis para várias tarefas de processamento de linguagem natural, como comparação de semelhança de texto, agrupamento e classificação.

In [11]:
type(gerador_embeddings)

langchain_community.embeddings.openai.OpenAIEmbeddings

## Testando os Vetores de Embeddings

In [12]:
# Testando o gerador de embeddings
vetores = gerador_embeddings.embed_query('Qual habilidade mais importante na era da IA?')

In [13]:
type(vetores)

list

In [14]:
vetores[1:10]

[-0.00252506310742038,
 0.019988425955745487,
 -0.01234033456857347,
 -0.0014928358571401482,
 0.021698311766916074,
 -0.0206379172488283,
 0.0009568398347858864,
 -0.006074069234811833,
 -0.023673294810624815]

## Definindo o Vector Store

https://www.trychroma.com/

In [15]:
# Definindo o nome do índice
index_name = 'dsa-index'

In [16]:
# Cria o vector store
index = Chroma.from_documents(dsa_doc, gerador_embeddings, collection_name = index_name)

In [17]:
type(index)

langchain_community.vectorstores.chroma.Chroma

Veja mais detalhes sobre o Chroma no videobook do Capítulo 5 do curso.

In [18]:
# Define a função de busca por similaridade
def dsa_busca_similaridade(query, k = 2):
    
    # Utiliza o método 'similarity_search' do objeto 'index' para buscar os 'k' resultados 
    # mais semelhantes à 'query'
    matching_results = index.similarity_search(query, k)
    
    # Retorna os resultados correspondentes da busca de similaridade
    return matching_results

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
## Criando a App do Assistente Pessoal com LangChain

In [19]:
# Cria instância do LLM
llm_dsa = OpenAI(openai_api_key = os.environ['OPENAI_API_KEY'], temperature = 0.3)

https://python.langchain.com/docs/modules/chains/document/stuff

https://js.langchain.com/docs/use_cases/question_answering/

In [20]:
# Cria a chain para perguntas e respostas em documentos
chain = load_qa_chain(llm_dsa, chain_type = 'stuff')

In [21]:
# Define a função para obter resposta
def obter_resposta(query):

    # Chama a função 'dsa_busca_similaridade' com a 'query' fornecida e armazena o resultado em 'doc_search'
    doc_search = dsa_busca_similaridade(query)
        
    # Utiliza o objeto 'chain' para executar a função run e processar a 'query' e os documentos encontrados, 
    # armazenando a resposta em 'response'
    response = chain.run(input_documents = doc_search, question = query)
    
    # Retorna a resposta obtida do processamento anterior
    return response

## Executando o Assistente de IA e Conversando com PDFs

In [22]:
# Pergunta para o arquivo pdf
dsa_prompt1 = "O que a pesquisa recente da Salesforce descobriu?"

In [23]:
# Obtém a resposta
resposta1 = obter_resposta(dsa_prompt1)

In [24]:
print(resposta1)

 A pesquisa recente da Salesforce descobriu que três quartos dos trabalhadores do mundo se sentem despreparados para os empregos que podem encontrar do outro lado do marco em que o número de horas trabalhadas pelas máquinas será igual ao número de horas trabalhadas pelos humanos.


In [25]:
# Pergunta para o arquivo pdf
dsa_prompt2 = "Qual percentual de empregadores estão tendo problemas para preencher vagas digitais com candidatos qualificados?"

In [26]:
# Obtém a resposta
resposta2 = obter_resposta(dsa_prompt2)

In [27]:
print(resposta2)

 Quase 60%.


In [28]:
# Pergunta para o arquivo pdf
dsa_prompt3 = "Qual a habilidade mais importante na era da Inteligência Artificial?"

In [29]:
# Obtém a resposta
resposta3 = obter_resposta(dsa_prompt3)

In [30]:
print(resposta3)

 "Seja Bom em Aprender".


In [31]:
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [32]:
#%watermark -v -m

In [33]:
#%watermark --iversions

# Fim